In [1]:
!pip install torchcodec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.7 MB/s eta 0:00:00


In [2]:
!git clone https://github.com/CS-3331-Fundamental-of-AI/WorldModel-Self-Driving-Car.git

Cloning into 'WorldModel-Self-Driving-Car'...
remote: Enumerating objects: 1287, done.
remote: Counting objects: 100% (190/190), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 1287 (delta 137), reused 117 (delta 67), pack-reused 1097 (from 1)
Receiving objects: 100% (1287/1287), 19.41 MiB | 35.18 MiB/s, done.
Resolving deltas: 100% (868/868), done.


In [3]:
import torch
from transformers import AutoVideoProcessor, AutoModel

processor = AutoVideoProcessor.from_pretrained(
    "facebook/vjepa2-vitl-fpc64-256"
)

model = AutoModel.from_pretrained(
    "facebook/vjepa2-vitl-fpc64-256",
    torch_dtype=torch.float16,     # CRITICAL
    device_map="cuda",             # force GPU
)

model.eval()
for p in model.parameters():
    p.requires_grad = False

2025-12-21 06:48:15.613102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766299695.818921      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766299695.875667      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766299696.391210      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299696.391253      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766299696.391256      24 computation_placer.cc:177] computation placer alr

video_preprocessor_config.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/785 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

In [4]:
# import kagglehub

# # Download latest version
# path = kagglehub.dataset_download("min1124/a-crude-data-set-converted-from-nuscene")

# print("Path to dataset files:", path)

In [5]:
# import os

# dataset_path = '/root/.cache/kagglehub/datasets/min1124/a-crude-data-set-converted-from-nuscene/versions/8'
# current_working_directory = os.getcwd()

# # The base name of the dataset directory to be copied
# dir_name = os.path.basename(dataset_path)

# # Construct the destination path
# destination_path = os.path.join(current_working_directory, dir_name)

# # Copy the directory
# !cp -r "{dataset_path}" "{destination_path}"

# print(f"Dataset copied from {dataset_path} to {destination_path}")

## Dataset Preparation

In [6]:
import os
import pandas as pd

DATA_ROOT = "/kaggle/input/a-crude-data-set-converted-from-nuscene/local_maps"

def build_map_dataframe(root):
    files = sorted([
        f for f in os.listdir(root)
        if f.endswith(".png")
    ])

    df = pd.DataFrame({
        "path": [os.path.join(root, f) for f in files],
        "name": files,
    })

    # optional: numeric id if useful
    df["map_id"] = df["name"].str.extract(r"(\d+)").astype(int)

    return df

df = build_map_dataframe(DATA_ROOT)
df.head()

,path,name,map_id
0,/kaggle/input/a-crude-data-set-converted-from-...,map_0000.png,0
1,/kaggle/input/a-crude-data-set-converted-from-...,map_0001.png,1
2,/kaggle/input/a-crude-data-set-converted-from-...,map_0002.png,2
3,/kaggle/input/a-crude-data-set-converted-from-...,map_0003.png,3
4,/kaggle/input/a-crude-data-set-converted-from-...,map_0004.png,4


In [7]:
from torch.utils.data import Dataset
import torchvision.transforms as T
from PIL import Image

class MapDataset(Dataset):
    def __init__(self, df, image_size=256, patch_size=16, mask_ratio=0.5):
        self.df = df
        self.image_size = image_size
        self.patch_size = patch_size
        self.mask_ratio = mask_ratio
        self.Hp = image_size // patch_size
        self.Wp = image_size // patch_size

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["path"]).convert("RGB")
        img = img.resize((self.image_size, self.image_size), Image.NEAREST)

        # uniform random patch mask
        N = self.Hp * self.Wp
        num_mask = int(self.mask_ratio * N)
        mask = torch.zeros(N, dtype=torch.bool)
        mask[torch.randperm(N)[:num_mask]] = True
        patch_mask = mask.view(self.Hp, self.Wp)

        return {
            "image": img,
            "patch_mask": patch_mask
        }

In [8]:
def collate_maps(batch):
    images = [item["image"] for item in batch]
    patch_masks = torch.stack([item["patch_mask"] for item in batch])

    return {
        "image": images,
        "patch_mask": patch_masks
    }

In [9]:
from PIL import ImageDraw

def patch_mask_to_token_mask(patch_mask, has_cls=False):
    """
    Spatial-only V-JEPA-2 masking (T = 1)

    patch_mask: [B, Hp, Wp]  (True = masked)
    returns:    [B, N]       where N = Hp * Wp
    """
    B, Hp, Wp = patch_mask.shape

    token_mask = patch_mask.reshape(B, Hp * Wp).bool()

    if has_cls:
        cls = torch.zeros(B, 1, dtype=torch.bool, device=patch_mask.device)
        token_mask = torch.cat([cls, token_mask], dim=1)

    return token_mask

def boolean_to_index_masks(bool_mask, pad_value=0):
    """
    bool_mask: [B, N] True = masked
    """
    B, N = bool_mask.shape
    context_list, target_list = [], []

    for b in range(B):
        target = torch.nonzero(bool_mask[b], as_tuple=False).flatten()
        context = torch.nonzero(~bool_mask[b], as_tuple=False).flatten()
        context_list.append(context)
        target_list.append(target)

    def pad(idxs):
        max_len = max(i.numel() for i in idxs)
        max_len = max(max_len, 1)  # avoid zero-length
        padded, valid = [], []

        for i in idxs:
            L = i.numel()
            pad_len = max_len - L
            padded.append(
                torch.cat([i, torch.full((pad_len,), pad_value, device=i.device)])
            )
            valid.append(
                torch.cat([torch.ones(L, dtype=torch.bool, device=i.device),
                           torch.zeros(pad_len, dtype=torch.bool, device=i.device)])
            )
        return torch.stack(padded), torch.stack(valid)

    context_idx, context_valid = pad(context_list)
    target_idx, target_valid   = pad(target_list)

    # safety clamp
    context_idx = context_idx.clamp(0, N - 1)
    target_idx  = target_idx.clamp(0, N - 1)

    return context_idx.long(), target_idx.long(), context_valid, target_valid


def mask_image(img, patch_mask, patch_size=16):
    img = img.copy()
    draw = ImageDraw.Draw(img)
    Hp, Wp = patch_mask.shape

    for y in range(Hp):
        for x in range(Wp):
            if patch_mask[y, x]:
                x0, y0 = x * patch_size, y * patch_size
                draw.rectangle(
                    [x0, y0, x0 + patch_size, y0 + patch_size],
                    fill=(127,127,127)
                )
    return img

In [10]:
from sklearn.model_selection import train_test_split

# Train-Val Split
train_df, val_df = train_test_split(
    df,
    test_size=0.1,
    random_state=42,
    shuffle=True,
)

train_df = train_df.reset_index(drop=True)
val_df   = val_df.reset_index(drop=True)

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    MapDataset(train_df, mask_ratio=0.3),
    batch_size=8,
    shuffle=True,
    num_workers=2,
    collate_fn=collate_maps,                 # ✅ REQUIRED
    pin_memory=False,
)

val_loader = DataLoader(
    MapDataset(val_df, mask_ratio=0.1),
    batch_size=8,
    shuffle=False,
    collate_fn=collate_maps,                 # ✅ REQUIRED
    num_workers=2,
)

In [12]:
import copy
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Create EMA (teacher) model
ema_model = copy.deepcopy(model)
ema_model.to(device)

# EMA model is always in eval mode
ema_model.eval()

# Disable gradients for EMA model
for p in ema_model.parameters():
    p.requires_grad = False

In [13]:
@torch.no_grad()
def update_ema(student, teacher, tau=0.996):
    for ps, pt in zip(student.parameters(), teacher.parameters()):
        pt.data.mul_(tau).add_(ps.data, alpha=1.0 - tau)
        
class TokenProjector(nn.Module):
    def __init__(self, in_dim=1024, out_dim=128):
        super().__init__()
        self.proj = nn.Linear(in_dim, out_dim)

    def forward(self, z):
        return self.proj(z)

class SpatialPredictorCNN(nn.Module):
    def __init__(self, embed_dim=128):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(embed_dim, embed_dim, 3, padding=1),
            nn.GELU(),
            nn.Conv2d(embed_dim, embed_dim, 3, padding=1),
            nn.GELU(),
            nn.Conv2d(embed_dim, embed_dim, 1),
        )

    def forward(self, x):
        return self.conv(x)

In [14]:
class PrimitiveLayerJEPA(nn.Module):
    def __init__(
        self,
        encoder,        # frozen V-JEPA-2 encoder
        grid_h=16,
        grid_w=16,
        enc_dim=1024,
        prim_dim=128
    ):
        super().__init__()
        self.encoder = encoder
        self.grid_h = grid_h
        self.grid_w = grid_w

        # 🔑 Projection: 1024 → 128
        self.project = nn.Linear(enc_dim, prim_dim)

        # Spatial predictor
        self.predictor = SpatialPredictorCNN(embed_dim=prim_dim)

    def forward(self, pixel_values):
        # -----------------------------
        # 1) Encoder (frozen)
        # -----------------------------
        with torch.no_grad():
            z = self.encoder(
                pixel_values_videos=pixel_values
            ).last_hidden_state  # [B, N, 1024]

        # -----------------------------
        # 2) Project to primitive dim
        # -----------------------------
        z = z.float()                    # FP32 safety
        z_proj = self.project(z)         # [B, N, 128]

        # -----------------------------
        # 3) Reshape to grid
        # -----------------------------
        B, N, D = z_proj.shape
        x = z_proj.transpose(1, 2).reshape(
            B, D, self.grid_h, self.grid_w
        )

        # -----------------------------
        # 4) Spatial predictor
        # -----------------------------
        delta = self.predictor(x)

        # -----------------------------
        # 5) Back to tokens
        # -----------------------------
        z_hat = delta.reshape(B, D, N).transpose(1, 2)

        return z_hat, z_proj

In [15]:
import torch
import torch.nn.functional as F


def jepa_embedding_loss(
    z_hat,
    z_target,
    alpha=1.0,   # alignment weight
    beta=1.0,    # variance weight
    gamma=0.1,   # covariance weight
    eps=1e-4,
):
    """
    JEPA / VICReg-style embedding loss.

    Args:
        z_hat:    [B, N, D] predicted embeddings
        z_target: [B, N, D] target embeddings (stop-grad or frozen encoder)
    Returns:
        total_loss, loss_dict
    """

    # --------------------------------------------------
    # 1) Alignment loss (JEPA core)
    # --------------------------------------------------
    loss_align = F.l1_loss(z_hat, z_target)

    # --------------------------------------------------
    # 2) Variance loss (anti-collapse)
    # --------------------------------------------------
    def variance_loss(z):
        # flatten tokens: [B*N, D]
        z = z.reshape(-1, z.size(-1))
        std = torch.sqrt(z.var(dim=0, unbiased=False) + eps)
        return torch.mean(F.relu(1.0 - std))

    loss_var = variance_loss(z_hat)

    # --------------------------------------------------
    # 3) Covariance loss (decorrelation)
    # --------------------------------------------------
    def covariance_loss(z):
        z = z.reshape(-1, z.size(-1))   # [B*N, D]
        z = z - z.mean(dim=0, keepdim=True)

        N = z.size(0)
        cov = (z.T @ z) / (N - 1)

        off_diag = cov - torch.diag(torch.diag(cov))
        return (off_diag ** 2).sum() / z.size(-1)

    loss_cov = covariance_loss(z_hat)

    # --------------------------------------------------
    # 4) Total loss
    # --------------------------------------------------
    total_loss = (
        alpha * loss_align
        + beta * loss_var
        + gamma * loss_cov
    )

    loss_dict = {
        "loss_total": total_loss,
        "loss_align": loss_align,
        "loss_var": loss_var,
        "loss_cov": loss_cov,
    }

    return total_loss, loss_dict

In [16]:
!pip install comet_ml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.1 MB/s eta 0:00:00
  Attempting uninstall: python-box
    Found existing installation: python-box 7.3.2
    Uninstalling python-box-7.3.2:
      Successfully uninstalled python-box-7.3.2


In [17]:
from comet_ml import Experiment
experiment = Experiment(
    api_key="Raze72byBUZdn5cWyLf3IFZ4h",
    project_name="vjepa2-primitive-layer",
    workspace="dtj-tran",
    auto_param_logging=False,
    auto_metric_logging=False,
)
experiment.log_parameters({
    "model": "vjepa2-vitl-fpc64-256",
    "encoder_frozen": True,
    "primitive_dim": 128,
    "grid_h": 16,
    "grid_w": 16,
    "loss": "L1",
    "optimizer": "AdamW",
    "lr": 1e-4,
    "weight_decay": 1e-4,
    "precision": "encoder_fp16_predictor_fp32",
})


COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: keras, tensorflow, sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/dtj-tran/vjepa2-primitive-layer/899911e6dc2d488eb35c3a94e05eb3b4



In [18]:
# --------------------------------------
# Freeze encoder
# --------------------------------------
for p in model.encoder.parameters():
    p.requires_grad = False
model.encoder.eval()

# --------------------------------------
# Primitive JEPA model
# --------------------------------------
prim_model = PrimitiveLayerJEPA(
    encoder=model.encoder,
    grid_h=16,
    grid_w=16,
    enc_dim=1024,
    prim_dim=128,
).to(device)

# Predictor MUST be FP32
prim_model.predictor.to(dtype=torch.float32)
prim_model.train()

optimizer = torch.optim.AdamW(
    prim_model.predictor.parameters(),
    lr=1e-4,
    weight_decay=1e-4,
)

In [19]:
global_step = 0
NUM_EPOCHS = 1

for epoch in range(NUM_EPOCHS):
    for step, batch in enumerate(train_loader):

        # --------------------------------------------------
        # 1) Prepare inputs
        # --------------------------------------------------
        images = [[img] for img in batch["image"]]

        px = processor(
            videos=images,
            return_tensors="pt"
        )["pixel_values_videos"].to(device, dtype=torch.float16)

        # --------------------------------------------------
        # 2) Forward (encoder frozen, predictor trainable)
        # --------------------------------------------------
        z_hat, z = prim_model(px)

        # --------------------------------------------------
        # 3) JEPA / VICReg-style loss (FP32)
        # --------------------------------------------------
        total_loss, loss_dict = jepa_embedding_loss(
            z_hat.float(),
            z.float(),
            alpha=1.0,
            beta=1.0,
            gamma=0.1,
        )

        # --------------------------------------------------
        # 4) Backprop
        # --------------------------------------------------
        optimizer.zero_grad(set_to_none=True)
        total_loss.backward()
        optimizer.step()

        # --------------------------------------------------
        # 5) Comet-ML logging (losses)
        # --------------------------------------------------
        experiment.log_metrics({
            "train/loss_total": loss_dict["loss_total"].item(),
            "train/loss_align": loss_dict["loss_align"].item(),
            "train/loss_var":   loss_dict["loss_var"].item(),
            "train/loss_cov":   loss_dict["loss_cov"].item(),
        }, step=global_step)

        # --------------------------------------------------
        # 6) Comet-ML logging (stats)
        # --------------------------------------------------
        if global_step % 100 == 0:
            experiment.log_metrics({
                "stats/z_mean":      z.mean().item(),
                "stats/z_std":       z.std().item(),
                "stats/z_hat_mean":  z_hat.mean().item(),
                "stats/z_hat_std":   z_hat.std().item(),
            }, step=global_step)

        # --------------------------------------------------
        # 7) Console logging (optional)
        # --------------------------------------------------
        # if global_step % 50 == 0:
        #     print(
        #         f"[epoch {epoch:02d} | step {global_step:06d}] "
        #         f"total={loss_dict['loss_total'].item():.4f} "
        #         f"align={loss_dict['loss_align'].item():.4f} "
        #         f"var={loss_dict['loss_var'].item():.4f} "
        #         f"cov={loss_dict['loss_cov'].item():.4f}"
        #     )

        global_step += 1

    # --------------------------------------------------
    # 8) Epoch-level logging
    # --------------------------------------------------
    experiment.log_metric("epoch", epoch, step=global_step)

    if global_step % 500 == 0:
        experiment.log_histogram_3d(
            z_hat.detach().float().cpu().numpy(),
            name="z_hat_distribution",
            step=global_step,
        )